# TensorFlow Homework

## If you know Numpy*

In [11]:
import numpy as np
a = np.zeros((2,2)); b = np.ones((2,2))
print("np.sum = ", np.sum(b, axis = 1))
print("a shape = ",a.shape)
print("np.reshape = ", np.reshape(a,(1,4)))

np.sum =  [ 2.  2.]
a shape =  (2, 2)
np.reshape =  [[ 0.  0.  0.  0.]]


## Repeat in TensorFlow

In [16]:
import tensorflow as tf
sess = tf.Session()
a = tf.zeros((2,2));
b = tf.ones((2,2))
sess.run(tf.reduce_sum(b,axis = 1))
a.get_shape()
sess.run(tf.reshape(a, (1,4)))

array([[ 0.,  0.,  0.,  0.]], dtype=float32)

## sess.run()

In [20]:
sess = tf.Session()
a = np.zeros((2,2))
ta = tf.zeros((2,2))
print(a)
print(ta)
print(sess.run(ta))



[[ 0.  0.]
 [ 0.  0.]]
Tensor("zeros_6:0", shape=(2, 2), dtype=float32)
[[ 0.  0.]
 [ 0.  0.]]


## Computation Graph

In [21]:
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
sess = tf.Session()
print(sess.run(c))

30.0


## TensorFlow Session

In [23]:
sess = tf.Session()
print(sess.run(c))
with tf.Session() as sess : 
    print(sess.run(c))
    print(c.eval())

30.0
30.0
30.0


## TensorFlow Variables

In [26]:
w = tf.Variable(tf.zeros((2,2)), name = "weight")
with tf.Session() as sess :
    print(sess.run(w))

FailedPreconditionError: Attempting to use uninitialized value weight
	 [[Node: _send_weight_0 = _Send[T=DT_FLOAT, client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1355712482197775703, tensor_name="weight:0", _device="/job:localhost/replica:0/task:0/cpu:0"](weight)]]

## Variables need initialization!

In [27]:
w = tf.Variable(tf.random_normal([5,2], stddev = 0.1), name = "weight")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w))

[[ 0.0265967   0.04112726]
 [ 0.21413822  0.00034742]
 [ 0.14611542 -0.02241675]
 [ 0.11108724 -0.00270839]
 [-0.03209164 -0.02022278]]


## Updating Variables

In [29]:
state = tf.Variable(0, name = "counter")
new_value = tf.add(state, tf.constant(1))
update = tf.assign(state, new_value)

with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    print(sess.run(state))
    for _ in range(3) :
        sess.run(update)
        print(sess.run(state))

0
1
2
3


## Fetching Variables

In [32]:
x1 = tf.constant(1)
x2 = tf.constant(2)
x3 = tf.constant(3)
temp = tf.add(x2, x3)
mul = tf.mul(x1, temp)

with tf.Session() as sess:
    result1, result2 = sess.run([mul, temp])
    print(result1, result2)

AttributeError: module 'tensorflow' has no attribute 'mul'

### Fetching Variables : tensorflow 1.0.0 release notes

- tf.mul, tf.sub and tf.neg are deprecated in favor of tf.multiply, tf.subtract and tf.negative.

In [33]:
x1 = tf.constant(1)
x2 = tf.constant(2)
x3 = tf.constant(3)
temp = tf.add(x2, x3)
mul = tf.multiply(x1, temp) 

# need to replace tf.mul with tf. multipy, release 1.0.0

with tf.Session() as sess:
    result1, result2 = sess.run([mul, temp])
    print(result1, result2)

5 5


## TensorFlow Placeholder

In [36]:
a = tf.placeholder(tf.int16)
b = tf.placeholder(tf.int16)
add = tf.add(a,b)
mul = tf.multiply(a,b)
with tf.Session() as sess : 
    print(sess.run(add, feed_dict = {a:2, b:3}))
    print(sess.run(mul, feed_dict = {a:2, b:3}))

5
6


In [49]:
#unsing tf.constant
matrix1 = tf.constant([[3.,3.]])
matrix2 = tf.constant([[2.], [2.]])
product = tf.matmul(matrix1, matrix2)

with tf.Session() as sess :
    result = sess.run(product)
    print(result)
    
#using placeholder

import numpy as np

matrix1 = tf.placeholder(tf.float32, [1,2])
matrix2 = tf.placeholder(tf.float32, [2,1])
product = tf.matmul(matrix1, matrix2)

with tf.Session() as sess:
    mv1 = np.array([[3., 3.]])
    mv2 = np.array([[2.], 2.])
    result = sess.run(product, feed_dict = {matrix1:mv1, matrix2:mv2})
    print(result)

[[ 12.]]


ValueError: setting an array element with a sequence.

## Example - MNIST with MLP

In [59]:
import tensorflow as tf

# Import MINST data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

learning_rate = 0.001
max_steps = 15000
batch_size = 128

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

def MLP(inputs) :
    
    w_1 = tf.Variable(tf.random_normal([784, 256]))
    b_1 = tf.Variable(tf.zeros([256]))
    
    w_2 = tf.Variable(tf.random_normal([256, 256]))
    b_2 = tf.Variable(tf.zeros([256]))
    
    w_out = tf.Variable(tf.random_normal([256, 10]))
    b_out = tf.Variable(tf.zeros([10]))
    
    h_1 = tf.add(tf.matmul(inputs, w_1), b_1)
    h_1 = tf.nn.relu(h_1)
    
    h_2 = tf.add(tf.matmul(h_1, w_2), b_2)
    h_2 = tf.nn.relu(h_2)
    
    out = tf.add(tf.matmul(h_2, w_out), b_out)
    
    return out

net = MLP(x)

# define loss and optimizer

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(net,y))

opt = tf.train.AdamOptimizer(learning_rate).minmize(loss_op)

# initializing the variables

init_op = tf.gloval_variables_initializer()

sess= tf.Session()
sess.run(init_op)

# train model

for step in range(max_steps) : 
    
    batch_x, batch_y = mnist.train.next_batch(batch_size) 
    loss = sess.run([opt, loss_op], feed_dict = {x:batch_x, y:batch_y})
    
    if (step+1) % 1000 == 0 :
        
        print("[{}/{}] loss:{:.3f}" .format(step+1, max_steps, loss))
        
print("Opimization Finished!")
        
#test model

correct_prediction = tf.equal(tf.argmax(net, 1), tf.argmax(y,1))

# calculate accuracy

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Train accuracy : {:.3f}" .format(sess.run(accuracy, feed_dict = {x : mnist.train.images, y : mnist.train.labels})))
print("Test accuracy : {:.3f}" .format(sess.run(accuracy, feed_dict = {x : mnist.test.images, y : mnist.test.labels})))


        
        
    
    

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz


ValueError: Only call `softmax_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)

## tf.variable_scope()

In [60]:
var1 = tf.Variable([1], name = "var")
with tf.variable_scope("foo") :
    with tf.variable_scope("bar") :
        var2 = tf.Variable([1], name = "var")
        var3 = tf.Variable([1], name = "var")
        
        print("var1: {}".format(var1.name))
        print("var2: {}".format(var2.name))
        print("var3: {}".format(var3.name))

var1: var:0
var2: foo/bar/var:0
var3: foo/bar/var_1:0


## tf.get_variable()

In [63]:
var1 = tf.Variable([1], name = "var")
with tf.variable_scope("foo") : 
    with tf.variable_scope("bar") as scp : 
        var2 = tf.Variable([1], name = "var")
        scp.reuse_variables()  # allow reuse variables
        var3 = tf.Variable([1], name = "var")
        
        print("var1: {}".format(var1.name))
        print("var2: {}".format(var2.name))
        print("var3: {}".format(var3.name))

var1: var_1:0
var2: foo_1/bar/var:0
var3: foo_1/bar/var_1:0


In [66]:
var1 = tf.get_variable("var", [1])
with tf.variable_scope("foo") : 
    with tf.variable_scope("bar") as scp :
        var2 = tf.get_variable("var", [1])
        scp.reuse_variables()
        var3 = tf.get_variable("var", [1])
        
        
                
print("var1: {}".format(var1.name))
print("var2: {}".format(var2.name))
print("var3: {}".format(var3.name))

ValueError: Variable var already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 1264, in __init__
    self._traceback = _extract_stack()
  File "C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 2395, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 763, in apply_op
    op_def=op_def)


- Variable var already exists!!!

## Parameter sharing

In [74]:
with tf.variable_scope("foo") :
    with tf.variable_scope("bar") as scp : 
        var11 = tf.get_variable("var",[1])
        scp.reuse_variables()
        var22 = tf.get_variable("var", [1])
        
        with tf.variable_scope("bar", reuse = True) :
            
            var33 = tf.get_variable("var",[1])
            
            
print("var: {}".format(var11.name))
print("var: {}".format(var22.name))
print("var: {}".format(var33.name))

ValueError: Variable foo/bar/var already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 1264, in __init__
    self._traceback = _extract_stack()
  File "C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 2395, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 763, in apply_op
    op_def=op_def)


## Wrappers

In [82]:
from tensorflow.contrib.layers import variance_scaling_initializer
he_init = variance_scaling_initializer()

def conv(bottom, num_filter, ksize =3, stride = 1, padding = "SAME", scope = None) :
    bottom_shape = bottom.get_shape().as_list()[3]
    
    with tf.variable_scope(scope or "conv") :
        
        w = tf.get_variable("w", [ksize, ksize, bottom_shape, num_filter], initializer=he_init)
        b = tf.get_variable("b", [num_filter], initializer = tf.constant_initialzer(0))
        
        x = tf.nn.conv2d(botto, w, strides = [1, stride, stride, 1], padding = padding)
        x = tf.nn.relu(tf.nn.bias_add(x,b))
        
        return x
    
    def maxpool(bottom, ksize = 2, stride = 2, padding = "SAME", scope = None) :
        
        with tf.variable_scope(scope or "maxpool") :
            pool = tf.nn.max_pool(bottom, ksize=[1, ksize, ksize, 1], strides=[1, stride, stride, 1], padding = padding)
            
            return pool
        
        def fc(bottom, num_dims, scope=None) :
            
            bottim_shape = bottom.get_shape().as_list()
            
            if len(bottom_shape) > 2:
                bottom =tf.reshape(bottom, [-1, reduce(lambda x, y: x*y, bottom_shape[1:])])
                bottom_shape = bottom.get_shape().as_list()
                
                with tf.variable_scope(scope or "fc") :
                    w = tf.get_variable("w", [bottom_shape[1], num_dims], initializer = he_init)
                    b = tf.get_variable("b", [num_dims], initializer = tf.constant_initializer(0))
                    
                    out = tf.nn.bias_add(tf.matmul(bottom, w), b)
                    
                    return out
                
                def fc_relu(bottom, num_dims, scope=None) :
                    
                    with tf.variable_scope(scope or "fc"):
                        out = fc(bottom, num_dims, scope = "fc")
                        relu = tf.nn.relu(out)
                        
                        return relu
                    
                    keep_prob = tf.placeholder(tf.float32, None)
                    
                    def conv_net(x, keep_prob) :
                        
                        x = tf.reshape(x, shape = [-1, 28, 28, 1])
                        
                        conv1 = conv(x, 32, 5, scope="conv_1")
                        conv1 = maxpool(conv1, scope="maxpool_1")
                        conv2 = conv(conv1, 64, 5, scope = "conv2")
                        conv2 = maxpool(conv2, scope = "maxpool_2")
                        
                        fc1 = fc_relu(conv2, 1024, scope = "fc_1")
                        fc1 = tf.nn.dropout(fc1, keep_prob)
                        
                        out = fc(fc1, 10, scope = "out")
                        
                        return out

## Session config

In [83]:
config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)